In [1]:
import pandas as pd
import numpy as np
import json

from contextualized_topic_models.models.ctm import ZeroShotTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing

import nltk

nltk.download('stopwords')

C:\Users\loren\anaconda3\envs\news\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\loren\anaconda3\envs\news\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\loren\anaconda3\envs\news\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\loren\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv("D:\commoncrawl\sample.csv")
df.dropna(inplace=True)
len(df)
# COLUMNS = 'title', 'maintext', 'date_publish', 'language', 'source_domain', 'id'

289518

In [3]:
with open(r"results\topic_dump.json", 'r') as json_file:
    topic_map = json.load(json_file)

In [4]:
with open(r"results\topics_layer.json", "r") as json_file:
    topics = json.load(json_file)
topics

{'0': ['celebrity',
  'island',
  'viewers',
  'instagram',
  'meghan',
  'prince',
  'revealed',
  'duchess',
  'wedding',
  'actress'],
 '1': ['win',
  'transfer',
  'summer',
  'side',
  'players',
  'season',
  'real',
  'world',
  'team',
  'club'],
 '2': ['get',
  'subscribe',
  'email',
  'try',
  'later',
  'notice',
  'thank',
  'could',
  'privacy',
  'invalid'],
 '3': ['labour',
  'prime',
  'party',
  'theresa',
  'trump',
  'minister',
  'eu',
  'mr',
  'president',
  'brexit'],
 '4': ['updates',
  'daily',
  'directly',
  'get',
  'inbox',
  'later',
  'try',
  'thank',
  'subscribe',
  'could']}

In [8]:
topic = 3
keys = [k for k, v in topic_map.items() if v == topic]
len(keys)

22449

In [9]:
tmp = df[df['id'].isin(keys)]
tmp = tmp[tmp['language'] == 'en']
tmp

,title,maintext,date_publish,language,source_domain,id
29272,Germany news: Economic crisis LOOMING over Tur...,The Turkish currency has lost nearly 40 percen...,2018-08-20 08:03:00,en,www.express.co.uk,e2fdaa000f4c0d53214064b04456dd20312303b2382f0b...
29340,'It's a TRAP' Brexiteer BRILLIANTLY dismantles...,"Lucy Harris, the founder of Brexit support gro...",2018-08-20 14:17:00,en,www.express.co.uk,3a7bd1039a9713103698605c54649771a2c8ef9a091aa0...
29691,Andy Murray eager to make most of Wimbledon re...,Andy Murray is plotting doubles success on his...,2019-07-03 16:52:00,en,www.express.co.uk,1226205a4aedee35b06ce9ad14435438176f000121bf40...
29797,World War 3 warning: NATO deploys reinforcemen...,Secretary General Jens Stoltenberg blamed Mosc...,2018-12-04 11:50:00,en,www.express.co.uk,85ca7b627696bc2f32f2900b5426ccad6314f02a2b3630...
29827,India suspends trade with Pakistan and blames ...,Indian officials are cracking down on trade be...,2019-04-19 15:54:00,en,www.express.co.uk,91f8e7148b44059ae2daa56c5045a453dabba9c18f876b...
...,...,...,...,...,...,...
427299,Fund manager Richard Buxton says secret Nissan...,The Japanese car maker was offered a secret pa...,2019-02-10 00:01:00,en,www.thetimes.co.uk,26a064e9f92309cff1497b8477ed1a5e0cdf1337b47b34...
427335,Time for social media to face up to the age of...,Social media companies are so adept at produci...,2019-02-10 00:01:00,en,www.thetimes.co.uk,f6a8d58e1cb8c1650698fe5a82c7a29a64266bfda78915...
427431,Dip in users takes Rightmove in wrong direction,The online property website had its bigeest st...,2019-03-01 12:00:00,en,www.thetimes.co.uk,39c6f2e5d51f9bfe5da1fdb4a285d217ebb0cfc976feb7...
427717,Picasso turns up 20 years after yacht theft,The recovery of the stolen Picasso was almost ...,2019-03-27 00:01:00,en,www.thetimes.co.uk,72cf68579f2d49e3c14a0f11608349a82fdf00c0529f8b...


In [11]:
for x in tmp.title[:50]:
    print(x)
    print("==============================================================")

Germany news: Economic crisis LOOMING over Turkey lira disaster - Merkel adviser
'It's a TRAP' Brexiteer BRILLIANTLY dismantles call for a 'people's vote' by ex-BBC host
Andy Murray eager to make most of Wimbledon return with doubles runs
World War 3 warning: NATO deploys reinforcements amid fears of fresh Russia conflict
India suspends trade with Pakistan and blames Islamabad for 'funnelling illegal weapons'
Alexander Zverev will lose coach Ivan Lendl if THIS happens - expert
Chelsea handed fresh transfer headache with chiefs already preparing for Eden Hazard exit
Brexit votes CHAOS: Theresa May resignation must happen 'QUICKLY' after deal failure
Why #DefendAfrin? Confronting authoritarian populism with radical democracy
With Tillerson in Latin America, Monroe is back
The new Brexit minister, the arms industry, the American hard right… and Equatorial Guinea
Invisible people: why Ukraine needs to take palliative care seriously
In Italy's centre-left, the hyphen is morphing into a wide